# IUM etap 1
## To jest brudnopis, potem to jakoś ładnie przepiszemy
###### Definicja problemu biznesowego
Na ten moment: “Niektórzy klienci nie mogą zdecydować się na zakup oglądając produkt na stronie –
ale pewnie gdybyśmy wiedzieli, które to są osoby, to odpowiednia zniżka skłoniłaby 
ich do zakupu"


###### Zadanie modelowania
1. Grupowanie klientów => Maksymalizacja przychodów w zależności od zniżki dla danej grupy? KOM: to trochę wychodzi poza zakes zadania
2. Binarna klasyfikacja czy sesja danej osoby zakończy się zakupem? 
3. Regresja określająca prawdopodobieństwo, że sesja zakończy się sukcesem? KOM: tutaj klient miałby duże pole manewru w przyznawaniu zniżek, czyli moglibyśmy wymagać więcej pieniędzy

###### Kryterium sukcesu

1. więcej kasy niż losowe(w proporcjach takich jak teraz) przyznawanie discountów?
2. większy procent poprawnej klasyfikacji niż założenie że wszyscy "nie kupią"(w łatwy sposób do policzenia na podstawie dostępnych danych)
3. trudno powiedzieć, może MSE mniejszy niż jakaś arbitralnie uzgodniona z klientem wartość?




## Analiza danych

In [20]:
import pandas as pd

In [21]:
#read all data
users = pd.read_json('dane/users.jsonl', lines=True)
products = pd.read_json('dane/products.jsonl', lines=True)
sessions = pd.read_json('dane/sessions.jsonl', lines=True)
deliveries = pd.read_json('dane/deliveries.jsonl', lines=True)

### Users.jsonl

In [22]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   user_id  200 non-null    int64 
 1   name     200 non-null    object
 2   city     200 non-null    object
 3   street   200 non-null    object
dtypes: int64(1), object(3)
memory usage: 6.4+ KB


In [24]:
users

,user_id,name,city,street
0,102,Ignacy Kozica,Police,ul. Wiśniowa 14
1,103,Sylwia Czubaszek,Warszawa,plac Towarowa 28
2,104,Bruno Bak,Kutno,ul. Bolesława Krzywoustego 91
3,105,Aniela Kaziród,Mielec,ul. Długa 69/74
4,106,Norbert Trybała,Warszawa,aleja Targowa 28/25
...,...,...,...,...
195,297,Justyna Grzelczyk,Konin,pl. Jaśminowa 73/03
196,298,Andrzej Wincenciak,Kutno,pl. Stycznia 46/82
197,299,Lidia Nestorowicz,Kutno,pl. Częstochowska 242
198,300,Borys Czupryna,Police,ul. Wodna 44/29


##### Dane w pliku **users.jsonl** są w poprawne, ale przydadzą nam się z niego tylko kolumny **user_id** oraz (może) **city**. Pozostałe są nieprzydatne.

### Products.jsonl

In [25]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 319 entries, 0 to 318
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   product_id     319 non-null    int64  
 1   product_name   319 non-null    object 
 2   category_path  319 non-null    object 
 3   price          319 non-null    float64
dtypes: float64(1), int64(1), object(2)
memory usage: 10.1+ KB


In [26]:
products

,product_id,product_name,category_path,price
0,1001,Telefon Siemens Gigaset DA310,Telefony i akcesoria;Telefony stacjonarne,58.97
1,1002,Kyocera FS-1135MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,2048.50
2,1003,Kyocera FS-3640MFP,Komputery;Drukarki i skanery;Biurowe urządzeni...,7639.00
3,1004,Fallout 3 (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
4,1005,Szalone Króliki Na żywo i w kolorze (Xbox 360),Gry i konsole;Gry na konsole;Gry Xbox 360,49.99
...,...,...,...,...
314,1315,Jabra Talk,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,54.99
315,1316,Plantronics Voyager Legend,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,249.00
316,1317,Plantronics Savi W740,Telefony i akcesoria;Akcesoria telefoniczne;Ze...,1303.96
317,1318,Plantronics Savi W710,Sprzęt RTV;Audio;Słuchawki,553.00


In [27]:
products['category_path'].unique()

array(['Telefony i akcesoria;Telefony stacjonarne',
       'Komputery;Drukarki i skanery;Biurowe urządzenia wielofunkcyjne',
       'Gry i konsole;Gry na konsole;Gry Xbox 360',
       'Komputery;Monitory;Monitory LCD',
       'Telefony i akcesoria;Telefony komórkowe',
       'Gry i konsole;Gry na konsole;Gry PlayStation3',
       'Gry i konsole;Gry komputerowe',
       'Telefony i akcesoria;Akcesoria telefoniczne;Zestawy głośnomówiące',
       'Sprzęt RTV;Video;Odtwarzacze DVD',
       'Komputery;Tablety i akcesoria;Tablety',
       'Sprzęt RTV;Przenośne audio i video;Odtwarzacze mp3 i mp4',
       'Sprzęt RTV;Video;Telewizory i akcesoria;Okulary 3D',
       'Sprzęt RTV;Video;Telewizory i akcesoria;Anteny RTV',
       'Telefony i akcesoria;Akcesoria telefoniczne;Zestawy słuchawkowe',
       'Sprzęt RTV;Audio;Słuchawki'], dtype=object)

Dane w **products.jsonl** wymagają przetworzenia:
- niektóre ceny są ujemne
- niektóre ceny są horrendalnie wysokie
- category_path należy odpowiednio przetworzyć

Wszystkie niepoprawne rekordy mogą zostać odrzucone z powodu względnie małej liczby(po ich usunięciu liczba dostępnych rekordów danych będzie wystarczająca). Po usunięciu niektórych wierszy powinniśmy nanieść odpowienie korekty do pozostałych ramek danych(tj. usunąć odpowiadające im wiersze)

### Sessions.jsonl

In [28]:
sessions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57650 entries, 0 to 57649
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   session_id        57650 non-null  int64         
 1   timestamp         57650 non-null  datetime64[ns]
 2   user_id           54795 non-null  float64       
 3   product_id        54763 non-null  float64       
 4   event_type        57650 non-null  object        
 5   offered_discount  57650 non-null  int64         
 6   purchase_id       3087 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(1)
memory usage: 3.1+ MB


In [29]:
sessions

,session_id,timestamp,user_id,product_id,event_type,offered_discount,purchase_id
0,100001,2021-01-05 15:51:40,102.0,1017.0,VIEW_PRODUCT,0,NaN
1,100001,2021-01-05 15:53:25,102.0,1025.0,VIEW_PRODUCT,0,NaN
2,100001,2021-01-05 15:58:08,102.0,1030.0,VIEW_PRODUCT,0,NaN
3,100001,2021-01-05 16:01:00,102.0,1032.0,VIEW_PRODUCT,0,NaN
4,100001,2021-01-05 16:02:18,102.0,1033.0,VIEW_PRODUCT,0,NaN
...,...,...,...,...,...,...,...
57645,110442,2021-01-11 19:44:56,301.0,1054.0,VIEW_PRODUCT,0,NaN
57646,110442,2021-01-11 19:47:31,301.0,1055.0,VIEW_PRODUCT,0,NaN
57647,110442,2021-01-11 19:48:27,301.0,1056.0,VIEW_PRODUCT,0,NaN
57648,110442,2021-01-11 19:48:45,301.0,1057.0,VIEW_PRODUCT,0,NaN


In [30]:
sessions['event_type'].unique()

array(['VIEW_PRODUCT', 'BUY_PRODUCT'], dtype=object)

Wymagają obróbki:
- user_id czasami jest nullem
- product_id czasami jest nullem
- purchase_id jest niepotrzebne bo nie używamy deliveries.jsonl


## Podsumowanie analizy danych

Co by się przydało:
- jak długo klient oglądał dany produkt? żeby móc "odsiać" przypadkowe kliknięcie(arbirtalnie poniżej 2 sekund)
- większa pula meta danych klientów? żeby móc zgrupowac klientów w sposób bardziej wyszukany niż po miescie
- ocena przedmiotu? żeby móc proponować najlepsze produkty(długoterminowy zysk) zamiast często kupowanych ale potencjalnie wadliwych(krótkowzroczne)
- ???

**deliveries.jsonl** nie jest potrzebne

## Obróbka danych(przy założeniu zwykłej klasyfikacji - za czym optuję)
Proponuję dane w **sessions.jsonl** przerobić w taki sposób, żeby "scalić" rekordy odpowiadające jednej sesji w taki sposób żeby rekord sesji posiadał:
- czas trwania sesji(zwyczajne `max[] i min[]` )
- zniżka z całej sesji(z tego co widzę zniżka jest na całą sesję a nie na poszczególne produkty)
- user_id
- czy sesja zakończyłą się zakupem
- średnia kwota przeglądanych produktów? może mediana? może oba?
- liczba zarobionych pieniędzy?
- najczęściej przeglądana kategoria? może dwie najczęstsze?


##### Następnie zmergowałbym te przerobione dane z innymi ramkami danych w taki sposób żeby mieć wszsytko w jednym miejscu, czyli jeden rekod miałby:
- wszystko co wyżej
- miasto użtykownika
- coś z produktami trzeba wymyśleć, proponuję zrobić kolumne binarną na każdy produkt(albo przynajmniej na każdą kategorię), to sięchyba nazywa `one hot encoding`


Z prduktami też możemy się pobawić
np. policzyć jakąś proporcje zakup/oglądanie, co by potem mogło się przełożyć na predykcje czy coś kupujemy
ale tak po zastanowieniu to raczej zbędne, algorytmy same zrobią coś podobnego(może nie w prost, ale efekt będzie podobny)


